In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from statsmodels.stats import diagnostic
import statsmodels.stats as st
from scipy import stats
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import OneHotEncoder
# from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve

from director_scrap import director_scrap

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Crew Database

Database of directors and writers associated to each film

In [90]:
df_crew = pd.read_table('data/crew.tsv')
df_crew.head(1)

,tconst,directors,writers
0,tt0000001,nm0005690,NaN


# Name Database

Database of names of people having contributed to the movie (director, writer, actor, cosplayer...)

In [91]:
df_name = pd.read_table('data/name.tsv')
df_name.head(1)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0072308,tt0053137,tt0031983,tt0050419"


# Database Title

Database of titles of movies' and many other informations

In [92]:
df_title = pd.read_table('data/title.tsv')
df_title.head(1)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"


# Database Imdb_Rating

Database of movie ratings

In [93]:
df_rating = pd.read_table('data/imdb_rating.tsv')
df_rating.head(1)

,tconst,averageRating,numVotes
0,tt0000001,5.7,2004


# Database Principals

Job of the person, title (if applicable) and character name

In [121]:
df_principals = pd.read_table('data/principals.tsv')
df_principals.head(1)

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"


In [136]:
df_p = df_principals.drop(['job', 'characters', 'ordering'], axis=1)
df_p.head(3)

,tconst,nconst,category
0,tt0000001,nm1588970,self
1,tt0000001,nm0005690,director
2,tt0000001,nm0374658,cinematographer


In [139]:
print(len(df_p))
df_p_drop = df_p.drop_duplicates(subset=['tconst', 'category'])
print(len(df_p_drop))
df_p_drop.head()

58969176
30259451


,tconst,nconst,category
0,tt0000001,nm1588970,self
1,tt0000001,nm0005690,director
2,tt0000001,nm0374658,cinematographer
3,tt0000002,nm0721526,director
4,tt0000002,nm1335271,composer


In [140]:
df_pp = df_p_drop.pivot(index='tconst', columns='category', values='nconst').reset_index()
df_pp.head()

category,tconst,actor,actress,archive_footage,archive_sound,cinematographer,composer,director,editor,producer,production_designer,self,writer
0,tt0000001,NaN,NaN,NaN,NaN,nm0374658,NaN,nm0005690,NaN,NaN,NaN,nm1588970,NaN
1,tt0000002,NaN,NaN,NaN,NaN,NaN,nm1335271,nm0721526,NaN,NaN,NaN,NaN,NaN
2,tt0000003,NaN,NaN,NaN,NaN,NaN,nm1335271,nm0721526,nm5442200,nm1770680,NaN,NaN,NaN
3,tt0000004,NaN,NaN,NaN,NaN,NaN,nm1335271,nm0721526,NaN,NaN,NaN,NaN,NaN
4,tt0000005,nm0443482,NaN,NaN,NaN,NaN,NaN,nm0005690,NaN,nm0249379,NaN,NaN,NaN


In [144]:
df_roles = df_pp[['tconst', 'composer', 'producer']]
df_roles.head()

category,tconst,composer,producer
0,tt0000001,NaN,NaN
1,tt0000002,nm1335271,NaN
2,tt0000003,nm1335271,nm1770680
3,tt0000004,nm1335271,NaN
4,tt0000005,NaN,nm0249379


In [148]:
df_co = df_roles.merge(df_name, left_on='composer', right_on='nconst', how='inner')
df_co = df_co[['tconst', 'producer', 'primaryName']].rename(columns={'primaryName': 'Composer'})
df_co.head(1)

,tconst,producer,Composer
0,tt0000002,NaN,Gaston Paulin


In [150]:
df_prod = df_co.merge(df_name, left_on='producer', right_on='nconst', how='inner')
df_prod = df_prod[['tconst', 'primaryName', 'Composer']].rename(columns={'primaryName': 'Producer'})
df_prod.head(1)

,tconst,Producer,Composer
0,tt0000003,Julien Pappé,Gaston Paulin


# Merging Step

Merge the above databases and keep only the useful informations 

In [151]:
df_crew_rating = df_crew.merge(df_rating, how='inner', on='tconst')
df_crew_rating.head(1)

,tconst,directors,writers,averageRating,numVotes
0,tt0000001,nm0005690,NaN,5.7,2004


In [152]:
df_cr_title = df_crew_rating.merge(df_title, how='inner', on='tconst')
df_cr_title.head(1)

,tconst,directors,writers,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,nm0005690,NaN,5.7,2004,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short"


In [153]:
df_team = df_cr_title.merge(df_prod, on='tconst', how='left')
df_team.head(1)

,tconst,directors,writers,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,Producer,Composer
0,tt0000001,nm0005690,NaN,5.7,2004,short,Carmencita,Carmencita,0.0,1894.0,NaN,1.0,"Documentary,Short",NaN,NaN


In [159]:
df_crt_dir = df_team.merge(df_name, left_on='directors', right_on='nconst', how='inner')
df_crt_dir.head(1)

,tconst,directors,writers,averageRating,numVotes,titleType,primaryTitle,originalTitle,isAdult,startYear,...,runtimeMinutes,genres,Producer,Composer,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,tt0000001,nm0005690,NaN,5.7,2004,short,Carmencita,Carmencita,0.0,1894.0,...,1.0,"Documentary,Short",NaN,NaN,nm0005690,William K.L. Dickson,1860.0,1935.0,"cinematographer,director,producer","tt1428455,tt0219560,tt0308254,tt1496763"


In [160]:
df_crtd = df_crt_dir.drop(['tconst', 'directors', 'numVotes', 'primaryProfession', 'isAdult'], axis=1)
df_crtd.head(1)

,writers,averageRating,titleType,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,genres,Producer,Composer,nconst,primaryName,birthYear,deathYear,knownForTitles
0,NaN,5.7,short,Carmencita,Carmencita,1894.0,NaN,1.0,"Documentary,Short",NaN,NaN,nm0005690,William K.L. Dickson,1860.0,1935.0,"tt1428455,tt0219560,tt0308254,tt1496763"


In [161]:
df_crtd = df_crtd.rename(columns={'primaryName': 'director', 'birthYear': 'dir_birth', 'deathYear': 'dir_death'
                                 , 'runtimeMinutes': 'runtime_min', 'averageRating': 'imdb_rating', 
                                  'knownForTitles': 'dir_known_titles'})
df_crtd.head(1)

,writers,imdb_rating,titleType,primaryTitle,originalTitle,startYear,endYear,runtime_min,genres,Producer,Composer,nconst,director,dir_birth,dir_death,dir_known_titles
0,NaN,5.7,short,Carmencita,Carmencita,1894.0,NaN,1.0,"Documentary,Short",NaN,NaN,nm0005690,William K.L. Dickson,1860.0,1935.0,"tt1428455,tt0219560,tt0308254,tt1496763"


In [170]:
df_fin = df_crtd.merge(df_name, left_on='writers', right_on='nconst', how='left')
df_fin.head(1)

,writers,imdb_rating,titleType,primaryTitle,originalTitle,startYear,endYear,runtime_min,genres,Producer,...,director,dir_birth,dir_death,dir_known_titles,nconst_y,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,NaN,5.7,short,Carmencita,Carmencita,1894.0,NaN,1.0,"Documentary,Short",NaN,...,William K.L. Dickson,1860.0,1935.0,"tt1428455,tt0219560,tt0308254,tt1496763",NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
df = df_fin.drop(['nconst_x', 'nconst_y', 'primaryProfession', 'writers'], axis=1)
df = df.rename(columns={'primaryName': 'writer', 'birthYear': 'writer_birth', 'deathYear': 'writer_death',
                        'knownForTitles': 'writer_know_titles', 'titleType': 'type', 
                        'primaryTitle': 'popular_title', 'originalTitle': 'original_title',
                       'startYear': 'movie_start_year', 'endYear': 'movie_end_year'})
df.head(1)

,imdb_rating,type,popular_title,original_title,movie_start_year,movie_end_year,runtime_min,genres,Producer,Composer,director,dir_birth,dir_death,dir_known_titles,writer,writer_birth,writer_death,writer_know_titles
0,5.7,short,Carmencita,Carmencita,1894.0,NaN,1.0,"Documentary,Short",NaN,NaN,William K.L. Dickson,1860.0,1935.0,"tt1428455,tt0219560,tt0308254,tt1496763",NaN,NaN,NaN,NaN


In [172]:
print(len(df))
df = df.drop_duplicates()
print(len(df))

970351
970309


In [173]:
df = df.drop_duplicates(subset=['popular_title', 'movie_start_year'])
len(df)

910697

In [174]:
df.head(1)

,imdb_rating,type,popular_title,original_title,movie_start_year,movie_end_year,runtime_min,genres,Producer,Composer,director,dir_birth,dir_death,dir_known_titles,writer,writer_birth,writer_death,writer_know_titles
0,5.7,short,Carmencita,Carmencita,1894.0,NaN,1.0,"Documentary,Short",NaN,NaN,William K.L. Dickson,1860.0,1935.0,"tt1428455,tt0219560,tt0308254,tt1496763",NaN,NaN,NaN,NaN


# Export Database to tsv

**Dont run the below cell except if you want to modify the imdb_data.tsv file**

In [175]:
df.to_csv('data/imdb_dataV2.tsv', sep='\t', index=False)